## Data flow to mysql

In [1]:
import mysql.connector as mdb
from mysql.connector import Error
import sys, traceback

In [2]:
def open_database(host, db_name, username, password):
    try:
        connection = mdb.connect(host=host,
                                 database=db_name,
                                 user=username,
                                 password=password)
        if connection.is_connected():
           db_Info = connection.get_server_info()
           print("Connected to MySQL database... MySQL Server version on ",db_Info)
           cursor = connection.cursor()
           cursor.execute("select database();")
           record = cursor.fetchone()
           print ("Your connected to - ", record)
           cursor.close()
    except Error as e :
        print ("Error while connecting to MySQL", e)
    return connection

In [3]:
def close_database(connection):
    #closing database connection.
    if(connection.is_connected()):
        connection.close()
        print("MySQL connection is closed")

In [51]:
conn = open_database('localhost', 'caselaw', 'root', 'H3rnandez!')

Connected to MySQL database... MySQL Server version on  8.0.15
Your connected to -  ('caselaw',)


In [30]:
query = """SELECT * FROM caselaw.case LIMIT 10;"""
cursor = conn.cursor()
print(cursor.execute(query))

None


---
### Datasets

In [4]:
import pandas as pd
import numpy as np

In [5]:
!ls ../data_dump

case.csv                               countries.csv
case_opinion_from_advocate_general.csv legal_intelligence_cases.csv
caselaw_citations.csv                  legislation_citations.csv
cases_metadata.csv


In [48]:
df_case = pd.read_csv("../data_dump/case.csv").replace(np.nan, 'NULL', regex=True)
df_country = pd.read_csv("../data_dump/countries.csv").replace(np.nan, 'NULL', regex=True)
df_li_cases = pd.read_csv("../data_dump/legal_intelligence_cases.csv").replace(np.nan, 'NULL', regex=True)
df_case_opinion = pd.read_csv("../data_dump/case_opinion_from_advocate_general.csv").replace(np.nan, 'NULL', regex=True)
df_case_citation = pd.read_csv("../data_dump/caselaw_citations.csv").replace(np.nan, 'NULL', regex=True)
df_legislation_citation = pd.read_csv("../data_dump/legislation_citations.csv").replace(np.nan, 'NULL', regex=True)

In [105]:
df_case_opinion.head()

,case_id,date,case_number,description,language,venue,abstract,procedure_type,lodge_date,country,...,abstract.1,procedure_type.1,lodge_date.1,country.1,subject.1,authority.1,legal_references.1,related_cases.1,alternative_sources.1,full_text


## change LI Ecli to not underscores

 I leave this as a piece of code for now, but I guess we can simply save the updated li_dataframe so that we dont have to run this everytime. 

In [49]:
for i, li_case in df_li_cases.iterrows():
    #get the ecli number
    ecli = li_case.ecli
    #replace the underscores
    new_ecli = ecli.replace('_', ':')
    #save the new ecli in the dataframe
    df_li_cases.at[i, 'ecli'] = new_ecli

---
### Sample to run the procedure

In [93]:
sample_size = 10000
#list of ecli numbers we choose for our sample
sample = list(df_case['case_id'].sample(n=sample_size, random_state=8))

In [78]:
#compare case opinion and cases on ecli cause it seems like there is no overlap 
#case_eclis = df_case['case_id'].values
#opinion_eclis = df_case_opinion['case_id'].values

In [98]:
##0: case sample
df_case_sp = df_case[df_case['case_id'].isin(sample)]
len(df_case_sp)

10000

In [94]:
##1: legislation citations sample
df_legislation_citation_sp = df_legislation_citation[df_legislation_citation['source_ecli'].isin(sample)]
len(df_legislation_citation_sp)

134

In [95]:
##2: case citations sample
df_case_citation_sp = df_case_citation[df_case_citation['source_ecli'].isin(sample)]
len(df_case_citation_sp)

2895

In [96]:
##3: case opinions sample
df_case_opinion_sp = df_case_opinion[df_case_opinion['case_id'].isin(sample)]
len(df_case_opinion_sp)

0

In [97]:
##4: li cases sample
df_li_cases_sp = df_li_cases[df_li_cases['ecli'].isin(sample)]
len(df_li_cases_sp)

46

In [99]:
#change names 
df_case = df_case_sp
df_legislation_citation = df_legislation_citation_sp
df_case_citation = df_case_citation_sp
df_case_opinion = df_case_opinion_sp
df_li_cases = df_li_cases_sp


---
### Utils

In [33]:
def to_tuples(df):
    return [tuple(x) for x in df.values]

In [54]:
def clean_table_sql(table_name):
    try:
        cursor1 = conn.cursor(buffered=True)
        query1 = """DELETE FROM `{}`;""".format(table_name)
        cursor1.execute(query1)
        cursor2 = conn.cursor(buffered=True)
        query2 = """ALTER TABLE `{}` AUTO_INCREMENT = 1;""".format(table_name)
        cursor2.execute(query2)
        conn.commit()
    except Error as error :
        conn.rollback()
        print("Failed to delete MySQL table {}".format(error))
        traceback.print_exc(file=sys.stdout)

In [35]:
def get_parent_ids(df, columnname):    
    pid = []
    cursor = conn .cursor()
    for idx, data in enumerate(df[columnname]):
        if '"' in data:
            sql_select_Query = "SELECT id FROM court WHERE name = '{}'".format(data)
            datum = data
        else:
            sql_select_Query = "SELECT id FROM court WHERE name = \"{}\"".format(data)
            datum = data
        cursor.execute(sql_select_Query)
        records = cursor.fetchall()
        if(len(records) == 0):
            print('no records')
        #print(idx, records)
        #print(sql_select_Query)
        pid.append(records[0][0])
    cursor.close()
    return pid

---
### Tables Hierarchies

1. Court
2. Case
3. Case Opinion
4. LI Case
5. Subject
6. Case subject
7. Country
8. Case country
9. Case citation
10. Legislation citation
11. Case related decision


---
### Courts

In [36]:
court = pd.DataFrame()

In [37]:
courts_list = df_case.authority.unique()
#court['id'] = range(1, len(courts_list)+1)
court['name'] = courts_list

In [38]:
#dummies
court.loc[:,'type'] = 'NULL'
court.loc[:,'level'] = 'NULL'
#court.loc[:,'country'] = 'NULL' #VARCHAR(2)
#court.loc[:,'language'] = 'NULL' #VARCHAR(2)
court.loc[:,'jurisdiction'] = 'NULL' 
court.loc[:,'law_area'] = 'NULL'
court.loc[:,'authority_level'] = 'NULL'

In [39]:
court.head(2)

,name,type,level,jurisdiction,law_area,authority_level
0,Gerechtshof Amsterdam,NULL,NULL,NULL,NULL,NULL
1,Gerechtshof Arnhem,NULL,NULL,NULL,NULL,NULL


In [40]:
tuples = to_tuples(court)

In [55]:
clean_table_sql('court')

In [56]:
try:  
    cursor = conn.cursor()
    for idx, data in enumerate(tuples):
        if "'" in data[0]:
            query = """
            INSERT INTO `court` (name, type, level, jurisdiction, law_area, authority_level)
            VALUES (\"%s\", %s, %s, %s, %s, %s)
            """%data
        else:
            query = """
            INSERT INTO `court` (name, type, level, jurisdiction, law_area, authority_level)
            VALUES ('%s', %s, %s, %s, %s, %s)
            """%data
        #print(idx, query)
        result  = cursor.execute(query)
    conn.commit()
    cursor.close()
    print('Court table created')
except Error as error :
    conn.rollback()
    print("Failed to insert into MySQL table {}".format(error))
    traceback.print_exc(file=sys.stdout)

Court table created


In [57]:
len(court)

82

---
### Case

In [79]:
case = pd.DataFrame()
case_columns = ['date',
        'description',
        'language',
        'venue',
        'abstract',
        'procedure_type',
        'lodge_date',
        'link',
        'ecli']

In [80]:
temp = df_case[['date',
                'description',
                'language',
                'venue',
                'abstract',
                'procedure_type',
                'lodge_date',
                'alternative_sources',
                'case_id']]
temp.columns = case_columns
case = case.append(temp, ignore_index=True)

In [82]:
parents_ids = get_parent_ids(df_case, 'authority')
#around 20 mins

In [84]:
case['name'] = 'NULL'
case['court_id'] = parents_ids
case['date'] = pd.to_datetime(case['date'], errors='coerce')
case['date'] = [pd.to_datetime(i) if i != 'NULL' else pd.to_datetime('1900-01-01 00:00:00') for i in case['date']]
case['lodge_date'] = [pd.to_datetime(i) if i != 'NULL' else pd.to_datetime('1900-01-01 00:00:00') for i in case['lodge_date']]

In [85]:
#check this one
case['link'] = [i[0:250] for i in case['link']]

In [81]:
case.head()

,date,description,language,venue,abstract,procedure_type,lodge_date,link,ecli
0,1913-03-31,NULL,NL,NULL,NULL,NULL,NULL,"\n \n <rdf:list xmlns:rdf=""htt...",ECLI:NL:GHAMS:1913:15
1,1913-02-28,NULL,NL,NULL,NULL,NULL,NULL,"\n \n <rdf:list xmlns:rdf=""htt...",ECLI:NL:GHAMS:1913:28
2,1913-03-17,NULL,NL,NULL,NULL,NULL,NULL,"\n \n <rdf:list xmlns:rdf=""htt...",ECLI:NL:GHAMS:1913:19
3,1913-02-28,NULL,NL,NULL,NULL,NULL,NULL,"\n \n <rdf:list xmlns:rdf=""htt...",ECLI:NL:GHAMS:1913:4
4,1913-03-10,NULL,NL,NULL,NULL,NULL,NULL,"\n \n <rdf:list xmlns:rdf=""htt...",ECLI:NL:GHAMS:1913:20


In [86]:
tuples = to_tuples(case)

In [ ]:
try:  
    cursor = conn.cursor()
    for data in tuples:
        query = """INSERT INTO `case` (date, description, language, venue, abstract, procedure_type, lodge_date, link, ecli, name, court_id) 
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"""
        result  = cursor.execute(query, data)
    conn.commit()
    cursor.close()
    print('cases added')
except Error as error :
    conn.rollback()
    print("Failed to insert into MySQL table {}".format(error))
    traceback.print_exc(file=sys.stdout)

---
### Case opinion advocate general

In [94]:
case_opinion = pd.DataFrame()
case_opinion_columns = ['date',
                        'case_number',
                        'description',
                        'language',
                        'country',
                        'venue',
                        'abstract',
                        'procedure_type',
                        'authority',
                        'ecli']

In [95]:
temp = df_case_opinion[['date',
                        'case_number',
                        'description',
                        'language',
                        'country',
                        'venue',
                        'abstract',
                        'procedure_type',
                        'authority',
                        'case_id']]
temp.columns = case_opinion_columns
case_opinion = case_opinion.append(temp, ignore_index=True)

In [98]:
#case_opinion['date'] = pd.to_datetime(case_opinion['date'], errors='coerce')
case_opinion['date'] = [pd.to_datetime(i) if i != 'NULL' else pd.to_datetime('1900-01-01 00:00:00') for i in case_opinion['date']]

In [99]:
case_opinion.head(2)

,date,case_number,description,language,country,venue,abstract,procedure_type,authority,ecli
0,1913-04-18,x47,NULL,NL,NL,NULL,NULL,NULL,Parket bij de Hoge Raad,ECLI:NL:PHR:1913:AG1763
1,1913-04-18,x48,NULL,NL,NL,NULL,NULL,NULL,Parket bij de Hoge Raad,ECLI:NL:PHR:1913:AG1764


In [100]:
tuples = to_tuples(case_opinion)

In [101]:
try:  
    cursor = conn.cursor()
    for data in tuples:
        query = """INSERT INTO `case_opinion` (date, case_number, description, language, country, venue, abstract, procedure_type, authority, ecli) 
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"""
        result  = cursor.execute(query, data)
    conn.commit()
    cursor.close()
    print('case_opinion added')
except Error as error :
    conn.rollback()
    print("Failed to insert into MySQL table {}".format(error))
    traceback.print_exc(file=sys.stdout)

Failed to insert into MySQL table 1452 (23000): Cannot add or update a child row: a foreign key constraint fails (`caselaw`.`case_opinion`, CONSTRAINT `fk_case_opinion_case` FOREIGN KEY (`ecli`) REFERENCES `case` (`ecli`))
Traceback (most recent call last):
  File "/Users/pedrohserrano/opt/anaconda3/lib/python3.7/site-packages/mysql/connector/connection_cext.py", line 489, in cmd_query
    raw_as_string=raw_as_string)
_mysql_connector.MySQLInterfaceError: Cannot add or update a child row: a foreign key constraint fails (`caselaw`.`case_opinion`, CONSTRAINT `fk_case_opinion_case` FOREIGN KEY (`ecli`) REFERENCES `case` (`ecli`))

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "<ipython-input-101-8b6e007a7d2a>", line 6, in <module>
    result  = cursor.execute(query, data)
  File "/Users/pedrohserrano/opt/anaconda3/lib/python3.7/site-packages/mysql/connector/cursor_cext.py", line 266, in execute
    raw_as_string=self._raw_as

---
### Legal Intelligence Cases

In [101]:
legal_intelligence_case = pd.DataFrame()
legal_intelligence_case_columns = ['id',
           'ecli',
           'name',
           'date',
           'abstract',
           'subject',
           'link',
           'DisplayTitle',
           'OriginalUrl',
           'Jurisidiction',
           'DocumentType',
           'CaseNumber',
           'PublicationNumber',
           'IssueNumber',
           'lodge_date',
           'DateAdded',
           'Sources',
           'UrlWithAutoLogOnToken',
           'court',
           'DisplaySubtitle']

In [104]:
#li_cases['id'] = #do we create this one ourselves? 
#li_cases['ecli'] 
#li_cases['name'] 
#li_cases['date'] 
#li_cases['abstract']
#li_cases['subject']
#li_cases['link']
#li_cases['procedure_type']
#li_cases['court_id']
#li_cases['DisplayTitle']
#li_cases['OriginalUrl']
#li_cases['Jurisdiction']
#li_cases['DocumentType']
#li_cases['CaseNumber']
#li_cases['PublicationDate']
#li_cases['PublicationNumber']
#li_cases['IssueNumber']
#li_cases['DateAdded']
#li_cases['Sources']
#li_cases['UrlWithAutoLogOnToken']

legal_intelligence_case['date'] = [pd.to_datetime(i) if i != 'NULL' else pd.to_datetime('1900-01-01 00:00:00') for i in df_li_cases['date']]
legal_intelligence_case['lodge_date'] = [pd.to_datetime(i) if i != 'NULL' else pd.to_datetime('1900-01-01 00:00:00') for i in df_li_cases['lodge_date']]
legal_intelligence_case['DateAdded'] = [pd.to_datetime(i) if i != 'NULL' else pd.to_datetime('1900-01-01 00:00:00') for i in df_li_cases['DateAdded']]

In [ ]:
tuples = to_tuples(li_cases)
import traceback, sys
try:  
    cursor = conn.cursor()
    for data in tuples:
        query = """
                INSERT INTO legal_intelligence_case (id, ecli, name, date, abstract, subject, link, procedure_type, court_id, DisplayTitle, OriginalUrl, Jurisdiction, DocumentType, CaseNumber, PublicationDate, PublicationNumber, IssueNumber, DateAdded, Sources, UrlWithAutoLogOnToken)
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                """
        result  = cursor.execute(query, data)
    conn.commit()
    cursor.close()
    print('legal_intelligence_case added')
except Error as error :
    conn.rollback()
    print("Failed to insert into MySQL table {}".format(error))
    traceback.print_exc(file=sys.stdout)

---
### Subjects

In [11]:
subject = pd.DataFrame()
subject_columns = ['id',
           'name',
           'standard_name']

In [18]:
cases_metadata.subject.unique()[1:] #omit the null case

array(['Bestuursrecht', 'Bestuursrecht; Ambtenarenrecht',
       'Bestuursrecht; Socialezekerheidsrecht',
       'Bestuursrecht; Vreemdelingenrecht'], dtype=object)

In [23]:
#catalogue
subjects = pd.DataFrame()
subjects['name'] = cases_metadata.subject.unique()[1:] #omit the null case
#subjects['standard_name'] = cases_metadata.subject.unique()[1:]

In [105]:
#subjects.head()

In [24]:
tuples = to_tuples(subjects)
import traceback, sys
try:  
    cursor = conn.cursor()
    for data in tuples:
        query = """INSERT INTO `subjects` (name) 
        VALUES (%s)"""
        result  = cursor.execute(query, data)
    conn.commit()
    cursor.close()
    print('subjects added')
except Error as error :
    conn.rollback()
    print("Failed to insert into MySQL table {}".format(error))
    traceback.print_exc(file=sys.stdout)

subjects added


In [12]:
case_subject = pd.DataFrame()
case_subject_columns = ['case_id',
                'subject_id']

---
### Countries

In [13]:
country = pd.DataFrame()
country_columns = ['id',
           'name',
           'language',
           'flag',
           'eu',
           'eea']

In [11]:
#catalogue
#countries = pd.DataFrame()
#countries['country_id'] #NOT NULL COMMENT 'Country ISO Code',
#countries['name'] #'English name',
#countries['language'] #'ISO code for language spoken in country',
#countries['flag'] #'Link to image file of flag',
#countries['eu'] #'Part of EU or not?',
#countries['population'] #'Current / latest population number',
#countries['eea'] #'not sure what this stands for exactly...'

In [104]:
#countries.head()

In [14]:
tuples = to_tuples(countries)
import traceback, sys
try:  
    cursor = conn.cursor(buffered=True)
    for data in tuples:
        query = """INSERT INTO `countries` (country_id, eu, name, flag, population, eea) 
        VALUES (%s, %s, %s, %s, %s, %s)"""
        result  = cursor.execute(query, data)
    conn.commit()
    cursor.close()
    print('countries added')
except Error as error :
    conn.rollback()
    print("Failed to insert into MySQL table {}".format(error))
    traceback.print_exc(file=sys.stdout)

countries added


In [14]:
case_country = pd.DataFrame()
case_country_columns = ['case_id',
                'country_id']

---
### Case law citation

In [15]:
case_citation = pd.DataFrame()
case_citation_columns = ['id',
                 'source_ecli',
                 'source_paragraph',
                 'target_ecli',
                 'target_paragraph']

In [99]:
#one to many
#case_citation = pd.DataFrame()
#case_citation['citation_id'] #INT NOT NULL,
#case_citation['source_ecli'] #NOT NULL,`fk_source_case_cases`
#case_citation['source_paragraph'] #
#case_citation['target_ecli'] #NOT NULL,`fk_target_case_cases`
#case_citation['target_paragraph'] #

In [110]:
#case_citation.head()

In [102]:
tuples = to_tuples(case_citation)
import traceback, sys
try:  le
    cursor = conn.cursor()
    for data in tuples:
        query = """
                INSERT INTO case_citation (source_ecli, source_paragraph, target_ecli, target_paragraph)
                VALUES (%s, %s, %s, %s)
                """
        result  = cursor.execute(query, data)
    conn.commit()
    cursor.close()
    print('case_citation added')
except Error as error :
    conn.rollback()
    print("Failed to insert into MySQL table {}".format(error))
    traceback.print_exc(file=sys.stdout)

case_citation added


---
### Legislation citation

In [16]:
legislation_citation = pd.DataFrame()
legislation_citation_columns = ['id',
                        'source_paragraph',
                        'target_id',
                        'target_paragraph',
                        'target_name',
                        'target_sourcename',
                        'target_link',
                        'case_id']

In [94]:
#one to many 
#legislation_citation = pd.DataFrame()
#legislation_citation['citation_id'] #INT NOT NULL,
#legislation_citation['source_ecli'] #NOT NULL,`fk_source_case_legislation`
#legislation_citation['source_paragraph']
#legislation_citation['target_id'] #NOT NULL,
#legislation_citation['target_paragraph']
#legislation_citation['target_name']
#legislation_citation['target_sourcename']
#legislation_citation['target_link']

In [95]:
legislation_citation.rename(columns = {'target_article':'target_id',
                                       'target_article_paragraph':'target_paragraph',
                                       'target_article_webpage':'target_link'}, inplace=True)

In [109]:
#legislation_citation.head()

In [97]:
tuples = to_tuples(legislation_citation)
import traceback, sys
try:  
    cursor = conn.cursor()
    for data in tuples:
        query = """
                INSERT INTO legislation_citation (source_ecli, source_paragraph, target_id, target_paragraph, target_link)
                VALUES (%s, %s, %s, %s, %s)
                """
        result  = cursor.execute(query, data)
    conn.commit()
    cursor.close()
    print('legislation_citation added')
except Error as error :
    conn.rollback()
    print("Failed to insert into MySQL table {}".format(error))
    traceback.print_exc(file=sys.stdout)

legislation_citation added


---
### Case related decision

In [17]:
case_related_decision = pd.DataFrame()
case_related_decision_columns = ['id',
                'referencing_case_ecli',
                'referenced_case_ecli',
                'case_id']

In [63]:
#one to one
case_related_decision = pd.DataFrame()
case_related_decision['case_ecli'] = cases_metadata.id #NOT NULL,
case_related_decision['referencing_case_id'] = cases_metadata.related_cases#NOT NULL,
#case_related_decision['referenced_case_id']

In [107]:
#case_related_decision.head()

In [65]:
tuples = to_tuples(case_related_decision)
import traceback, sys
try:  
    cursor = conn.cursor()
    for data in tuples:
        query = """INSERT INTO `case_related_decision` (case_ecli, referencing_case_id) 
        VALUES (%s, %s)"""
        result  = cursor.execute(query, data)
    conn.commit()
    cursor.close()
    print('case_related_decision added')
except Error as error :
    conn.rollback()
    print("Failed to insert into MySQL table {}".format(error))
    traceback.print_exc(file=sys.stdout)

case_related_decision added
